In [6]:
# Скачаем файл содержащий целого ШЕКСПИРА
import wget
file_data = wget.download('https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt')

100% [..........................................................................] 1115394 / 1115394

In [7]:
# Прочитаем его чтобы взаимодействовать с ним
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [8]:
print(f"Длинна датасета в символах равна {len(text)}")

Длинна датасета в символах равна 1115394


In [9]:
# Посмотрим на первые 1000 знаков
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [10]:
# Тут все уникальные символы находящиеся в тексте 
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [13]:
# создадим словарь перевода символов в цифры
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # кодировщик: беруд строку, возвращает последовательность интов
decode = lambda l: ''.join([itos[i] for i in l]) # декодеровщик: берет последовательность интов и возвращает строку

print(encode("hii there"))
print(decode(encode("hii there")))

# поздравлаяю, мы получили простейший токенизатор. Но если что Open AI сделали свой TikTiken, чтобы токенезировать всё за нас

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [14]:
# Теперь закодируем весь датасет (текст) и сохраним его в виде torch.Tensor
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # наши ранее выведенные 1000 символов теперь выглядят вот так

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [15]:
# Теперь разобьем наш датасет на тренировочную и валидационные выборки
n = int(0.9*len(data)) # первые 90% будут тренировочные, отсальные валидационные
train_data = data[:n]
val_data = data[n:]

In [16]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [18]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Если на входе {context} то дальше должно быть: {target}")

Если на входе tensor([18]) то дальше должно быть: 47
Если на входе tensor([18, 47]) то дальше должно быть: 56
Если на входе tensor([18, 47, 56]) то дальше должно быть: 57
Если на входе tensor([18, 47, 56, 57]) то дальше должно быть: 58
Если на входе tensor([18, 47, 56, 57, 58]) то дальше должно быть: 1
Если на входе tensor([18, 47, 56, 57, 58,  1]) то дальше должно быть: 15
Если на входе tensor([18, 47, 56, 57, 58,  1, 15]) то дальше должно быть: 47
Если на входе tensor([18, 47, 56, 57, 58,  1, 15, 47]) то дальше должно быть: 58


In [21]:
torch.manual_seed(1337)
batch_size = 4 # Как много независимыз секменций(батчей) будут запущены паралельно?
block_size = 8 # Какая максимальная длинна контекста для предсказания?

def get_batch(split):
    # генерирует маленькие семенции (батчи) входов x и таргетов y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('На входе:')
print(xb.shape)
print(xb)
print('Таргеты:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # размер бача
    for t in range(block_size): # как далеко в контексты мы погружены
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"Когда на входе {context.tolist()} таргет: {target}")

На входе:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Таргеты:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
Когда на входе [24] таргет: 43
Когда на входе [24, 43] таргет: 58
Когда на входе [24, 43, 58] таргет: 5
Когда на входе [24, 43, 58, 5] таргет: 57
Когда на входе [24, 43, 58, 5, 57] таргет: 1
Когда на входе [24, 43, 58, 5, 57, 1] таргет: 46
Когда на входе [24, 43, 58, 5, 57, 1, 46] таргет: 43
Когда на входе [24, 43, 58, 5, 57, 1, 46, 43] таргет: 39
Когда на входе [44] таргет: 53
Когда на входе [44, 53] таргет: 56
Когда на входе [44, 53, 56] таргет: 1
Когда на входе [44, 53, 56, 1] таргет: 58
Когда на входе [44, 53, 56, 1, 58] таргет: 46
Когда на входе [44, 53, 56, 1, 58, 46] таргет: 39
Когда на

In [22]:
print(xb) # наш вход на трансформер

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [23]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # Каждый токен читается как вектор эмбендинга рамерностью в наш алфавит (vocab size)
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # Индексы и таргеты оба (B,T) вектора интов
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # индексы это (B, T) массив индикаторв текущего контекста
        for _ in range(max_new_tokens):
            # делаем предикт (LOGITS = предсказания)
            logits, loss = self(idx)
            # смотрим только на последний
            logits = logits[:, -1, :] # становится (B, C)
            # Применяем софтмакс и получаем вероятности
            probs = F.softmax(logits, dim=-1) # (B, C)
            # семплируем из распределения
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # добавляем семплированный индекс
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [24]:
# создадим оптимайзер
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [28]:
batch_size = 32
for steps in range(10000): # увеличим число шагов для хорошего результата...

    # разделим нашу дату
    xb, yb = get_batch('train')

    # и посчитаем ошибки
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.3624846935272217


In [29]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Tizearily t com: chavily; o fill wf mes co it hitodntly d d wsue ater to ovithA herisch ave une, t e pothamef. g st ofa nde hewhilero f tu dreanket iser oust he, s
s t,end ald atant

Turce dinsest; fowis MI ar gd yomblthaghrenenke this imy'dious bun histute ait MNow.
Shid thod thokedanenj's.
DUK: ishouthat,
's'd?
S:'lENoran igoventitean he, tt usWhyour ixxff ou blvelife eare mie bu pamak.
COnithololodaketwerothait g ESoco pe teran Gly? go!
Slirenkneverajest young y, cet l th e?
I he t ho jusofas


# Математический трюк 

In [39]:
# Игрушечный пример иллюстрирующий как умножение матриц 
# может использоваться для a "сбора весов"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [40]:
# Продолжая данный пример :

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [41]:
# мы хотим x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [42]:
# Версия 2: используем перемножение матриц для сбора весов
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [43]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [44]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [45]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [46]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [47]:
k.var()

tensor(1.0449)

In [48]:
q.var()

tensor(1.0700)

In [49]:
wei.var()

tensor(1.0918)

In [50]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [51]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [52]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [53]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [54]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))